In [2]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid
import os
from dotenv import load_dotenv

load_dotenv()  # Make sure this points to the correct path if .env isn't in the root directory

True

In [5]:
# To start off, you need to have created an account at https://kalshi.com (Production) 
# or an account on the Demo https://demo.kalshi.co/

prod_email = os.getenv("EMAIL") # change these to be your personal credentials
prod_password = os.getenv("PASSWORD")  # (for extra security, we recommend using a config file)

demo_email = "" # change these to be your personal credentials
demo_password = "" # (for extra security, we recommend using a config file)

# for prod
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in prod:
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

## if wanting to test in demo
# exchange_client = ExchangeClient(exchange_api_base = demo_api_base, email = demo_email, password = demo_password)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': False}


In [6]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {'limit':100,
                    'cursor':None, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker':None,
                    'max_close_ts':None, # pass in unix_ts
                    'min_close_ts':None, # pass in unix_ts
                    'status':None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in payload:', markets_response['markets'][0])
print()
print('cursor:', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in payload: {'ticker': 'RTCHRISTMASPAGEANT-85', 'event_ticker': 'RTCHRISTMASPAGEANT', 'market_type': 'binary', 'title': 'The Best Christmas Pageant Ever Rotten Tomatoes score?', 'subtitle': 'Above 85', 'yes_sub_title': 'Above 85', 'no_sub_title': 'Above 85', 'open_time': '2024-11-04T19:00:00Z', 'close_time': '2024-11-11T15:00:00Z', 'expected_expiration_time': '2024-11-11T15:00:00Z', 'expiration_time': '2024-11-18T15:00:00Z', 'latest_expiration_time': '2024-11-18T15:00:00Z', 'settlement_timer_seconds': 3598, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 84, 'yes_ask': 85, 'no_bid': 15, 'no_ask': 16, 'last_price': 84, 'previous_yes_bid': 72, 'previous_yes_ask': 81, 'previous_price': 71, 'volume': 3138, 'volume_24h': 5, 'liquidity': 713926, 'open_interest': 1196, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': 'Entertainment', 

In [8]:
# What are cursors and how do they work?
    
# The Cursor represents a pointer to the next page of records in the pagination.
# So this optional parameter, when filled, should be filled with the cursor string returned in a previous request to this end-point.
# Filling this would basically tell the api to get the next page containing the number of records passed on the limit parameter.
# On the other side not filling it tells the api you want to get the first page for another query.
# The cursor does not store any filters, so if any filter parameters like tickers, max_ts or min_ts were passed in the original query they must be passed again.

# Let's try it in action! Suppose we wanted to get the next 100 market objects...

market_params = {'limit':100,
                    'cursor':cursor, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker': None,
                    'max_close_ts': None, # pass in unix_ts
                    'min_close_ts': None, # pass in unix_ts
                    'status': None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']        

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in market_response payload:', markets_response['markets'][0]) # new markets!
print()
print('new cursor!', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in market_response payload: {'ticker': 'INX-24OCT23-B5762', 'event_ticker': 'INX-24OCT23', 'market_type': 'binary', 'title': 'Will the S&P 500 be between 5750 and 5774.99 at the end of Oct 23, 2024?', 'subtitle': '5,750 to 5,774.99', 'yes_sub_title': '5,750 to 5,774.99', 'no_sub_title': '5,750 to 5,774.99', 'open_time': '2024-10-23T06:00:00Z', 'close_time': '2024-10-23T20:00:00Z', 'expected_expiration_time': '2024-10-23T23:00:00Z', 'expiration_time': '2024-10-30T23:00:00Z', 'latest_expiration_time': '2024-10-30T23:00:00Z', 'settlement_timer_seconds': 3600, 'status': 'finalized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 5, 'no_bid': 95, 'no_ask': 100, 'last_price': 2, 'previous_yes_bid': 0, 'previous_yes_ask': 5, 'previous_price': 2, 'volume': 8362, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 1959, 'result': 'no', 'settlement_value': 0, 'can_close

In [54]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = markets_response['markets'][5]['event_ticker']
event_params = {'event_ticker': event_ticker}
event_response = exchange_client.get_event(**event_params)

print('keys:', event_response.keys())
print()
print('event object:', event_response['event'])
print()
print('first market in event_response payload:', event_response['markets'][0])

keys: dict_keys(['event', 'markets'])

event object: {'event_ticker': 'NASDAQ100-24OCT23', 'series_ticker': 'NASDAQ100', 'sub_title': 'On Oct 23, 2024', 'title': 'Nasdaq close price range on Oct 23, 2024?', 'mutually_exclusive': True, 'category': 'Financials', 'strike_date': '2024-10-23T20:00:00Z'}

first market in event_response payload: {'ticker': 'NASDAQ100-24OCT23-T18900', 'event_ticker': 'NASDAQ100-24OCT23', 'market_type': 'binary', 'title': '', 'subtitle': '18,899.99 or below', 'yes_sub_title': '18,899.99 or below', 'no_sub_title': '18,899.99 or below', 'open_time': '2024-10-23T06:00:00Z', 'close_time': '2024-10-23T20:00:00Z', 'expected_expiration_time': '2024-10-23T23:00:00Z', 'expiration_time': '2024-10-30T23:00:00Z', 'latest_expiration_time': '2024-10-30T23:00:00Z', 'settlement_timer_seconds': 3600, 'status': 'finalized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 100, 'no_bid': 0, 'no_ask': 100, 'last_price': 0, 'previo

In [55]:
# Next, let's look at series level data by passing a series ticker to the get_series endpoint! 
series_ticker = event_response['event']['series_ticker']
series_params = {'series_ticker': series_ticker}
series_response = exchange_client.get_series(**series_params)

print('keys:', series_response.keys())
print()
print('series object:', series_response['series'])
print()

keys: dict_keys(['series'])

series object: {'ticker': 'NASDAQ100', 'frequency': 'daily', 'title': 'Nasdaq range', 'category': 'Financials', 'tags': ['Nasdaq'], 'settlement_sources': [{'url': 'https://www.google.com/finance/quote/NDX:INDEXNASDAQ?hl=en', 'name': 'For example, Google Finance'}], 'contract_url': 'https://kalshi-public-docs.s3.us-east-1.amazonaws.com/regulatory/product-certifications/NASDAQ100.pdf'}



In [1]:
# Next let's look at the recent market history for a market (ENDPOINT DOESN'T WORK)
# ticker = 'FED-25DEC-T4.25'

# market_history_params = {'ticker': ticker,
#                             'limit': 100,
#                             'cursor': None,
#                             'max_ts': None, # pass in unix_ts
#                             'min_ts': round(time.time()-1000000) # passing a recent unix_ts
#                                 }
# market_history_response = exchange_client.get_market_history(**market_history_params)

# print('keys:', market_history_response.keys())
# print()
# print('most recent market history object:', market_history_response['history'][-1])
# print()

# # and then also look at the most current view of the orderbook
# market_history_params = {'ticker':ticker,
#                             'depth': 30
#                                 }
# orderbook_response = exchange_client.get_orderbook(**market_history_params)

# print('keys:', orderbook_response.keys())
# print()
# print('orderbook object:', orderbook_response)
# print()

In [15]:
# Now let's suppose we wanted to place a trade on one of these markets... 
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance

{'balance': 0}

In [16]:
# Now that you have some balance, you might want to see how your current positions are doing...

positions_params = {'limit': None,
                        'cursor': None,
                        'settlement_status': None,
                        'ticker': None,
                        'event_ticker': None}

current_position = exchange_client.get_positions(**positions_params)
current_position

{'cursor': '', 'market_positions': [], 'event_positions': []}

In [17]:
# seems like some of your recent orders had been filled. To check on those we use the get_positions endpoint ...

fills_params = {'ticker':None,
                    'order_id':None,
                    'min_ts':None,
                    'max_ts':None,
                    'limit': None,
                    'cursor': None}

fills = exchange_client.get_fills(**fills_params)
fills

{'fills': [], 'cursor': ''}

In [18]:
# you may even want to check on some of your recent positions settled...

settlement_params = {'limit': None,
                        'cursor': None}

settlements = exchange_client.get_portfolio_settlements(**settlement_params)
settlements

{'settlements': [], 'cursor': ''}